In [2]:
import requests
import pandas as pd

# Charger les résultats 2025
url = "https://api.jolpi.ca/ergast/f1/2025/results/?offset=120"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    races = data['MRData']['RaceTable']['Races']

    results = []
    for race in races:
        for result in race['Results']:
            results.append({
                'Round': int(race['round']),
                'Race': race['raceName'],
                'Date': race['date'],
                'Driver': result['Driver']['familyName'],
                'Constructor': result['Constructor']['name'],
                'Grid': int(result['grid']),
                'Position': int(result['position']),
                'Status': result['status'],
                'Points': float(result['points'])
            })

    df = pd.DataFrame(results)
    display(df)
else:
    raise Exception("Erreur lors de la récupération des données.")

,Round,Race,Date,Driver,Constructor,Grid,Position,Status,Points
0,7,Emilia Romagna Grand Prix,2025-05-18,Verstappen,Red Bull,2,1,Finished,25.0
1,7,Emilia Romagna Grand Prix,2025-05-18,Norris,McLaren,4,2,Finished,18.0
2,7,Emilia Romagna Grand Prix,2025-05-18,Piastri,McLaren,1,3,Finished,15.0
3,7,Emilia Romagna Grand Prix,2025-05-18,Hamilton,Ferrari,12,4,Finished,12.0
4,7,Emilia Romagna Grand Prix,2025-05-18,Albon,Williams,7,5,Finished,10.0
5,7,Emilia Romagna Grand Prix,2025-05-18,Leclerc,Ferrari,11,6,Finished,8.0
6,7,Emilia Romagna Grand Prix,2025-05-18,Russell,Mercedes,3,7,Finished,6.0
7,7,Emilia Romagna Grand Prix,2025-05-18,Sainz,Williams,6,8,Finished,4.0
8,7,Emilia Romagna Grand Prix,2025-05-18,Hadjar,RB F1 Team,9,9,Finished,2.0
9,7,Emilia Romagna Grand Prix,2025-05-18,Tsunoda,Red Bull,20,10,Finished,1.0


In [3]:
# Parse drivers
results = data['MRData']['RaceTable']['Races'][0]['Results']
driver_list = []

for r in results:
    driver = r['Driver']
    number = driver.get('permanentNumber', None)
    name = f"{driver['givenName']} {driver['familyName']}"
    driver_list.append({'Driver': name, 'Number': number})

df_drivers = pd.DataFrame(driver_list)
print(df_drivers)

                   Driver Number
0          Max Verstappen     33
1            Lando Norris      4
2           Oscar Piastri     81
3          Lewis Hamilton     44
4         Alexander Albon     23
5         Charles Leclerc     16
6          George Russell     63
7            Carlos Sainz     55
8            Isack Hadjar      6
9            Yuki Tsunoda     22
10        Fernando Alonso     14
11        Nico Hülkenberg     27
12           Pierre Gasly     10
13            Liam Lawson     30
14           Lance Stroll     18
15       Franco Colapinto     43
16         Oliver Bearman     87
17      Gabriel Bortoleto      5
18  Andrea Kimi Antonelli     12
19           Esteban Ocon     31


In [4]:
# On ne garde que les arrivées terminées
df = df[df['Status'] == 'Finished']

# Trier par numéro de manche
df.sort_values(by='Round', inplace=True)

In [5]:
# Fonction pour prédire la prochaine position en moyenne pondérée
def predict_position(driver_name, grid_pos, history_df):
    driver_data = history_df[history_df['Driver'] == driver_name]
    if len(driver_data) < 2:
        return grid_pos  # Si peu de données, on revient à la grille

    # Moyenne pondérée basée sur ses dernières courses
    weight = range(1, len(driver_data) + 1)
    weighted_avg = sum(p * w for p, w in zip(driver_data['Position'], weight)) / sum(weight)

    # Ajustement simple : on rapproche un peu de la grille
    return 0.7 * weighted_avg + 0.3 * grid_pos

In [6]:
rounds = sorted(df['Round'].unique())
last_round = rounds[-1]
df_train = df[df['Round'] < last_round]
df_test = df[df['Round'] == last_round]

# Prédictions
predictions = []

for _, row in df_test.iterrows():
    pred = predict_position(row['Driver'], row['Grid'], df_train)
    predictions.append(pred)

# Ajouter au DataFrame test
df_test = df_test.copy()
df_test['Predicted'] = predictions

# Affichage des 10 premiers résultats
print(df_test[['Driver', 'Grid', 'Position', 'Predicted']].sort_values('Predicted'))

        Driver  Grid  Position  Predicted
2      Piastri     1         3          1
0   Verstappen     2         1          2
6      Russell     3         7          3
1       Norris     4         2          4
10      Alonso     5        11          5
7        Sainz     6         8          6
4        Albon     7         5          7
14      Stroll     8        15          8
8       Hadjar     9         9          9
12       Gasly    10        13         10
5      Leclerc    11         6         11
3     Hamilton    12         4         12
17   Bortoleto    14        18         14
13      Lawson    15        14         15
15   Colapinto    16        16         16
11  Hülkenberg    17        12         17
16     Bearman    19        17         19
9      Tsunoda    20        10         20


In [7]:
df_test['Error'] = abs(df_test['Predicted'] - df_test['Position'])
mean_error = df_test['Error'].mean()
print(f"Erreur moyenne absolue : {mean_error:.2f} positions")

Erreur moyenne absolue : 3.56 positions
